# Python IUDX SDK - pyIUDX
In this notebook we will use the pyIUDX module to interact with the IUDX server to obtain information about the sensors connected to IUDX. The pyIUDX module provides APIs that simplify queries to the PUDX data exchange.

In the first part, we will query and obtain the sensor metadata (properties). This information is obtained by querying the catalogue server. The catalogue server provides all metadata associated with a sensor, including, sensor type, sensor location, sensor attributes, sensed quantities and their units etc.

In the second part, we will obtain sensed data associated with a set of sensors. 

## Install pyIUDX SDK module

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates
import folium

In [2]:
# install the latest version of pyIUDX module from github
pip install git+https://github.com/iudx/pyIUDX --upgrade

  Cloning https://github.com/iudx/pyIUDX to /tmp/pip-req-build-l_z2pdx_
  Running command git clone -q https://github.com/iudx/pyIUDX /tmp/pip-req-build-l_z2pdx_
  Created wheel for pyIUDX: filename=pyIUDX-0.0.1-cp36-none-any.whl size=21931 sha256=0cd5ab37f88c02d34cd01ebd5717e3060afe3e98272166ca3e236e311ce84ac5
  Stored in directory: /tmp/pip-ephem-wheel-cache-4uig8qco/wheels/9a/47/49/4628dcd08582cacb7599b922e10d77572044b4c49fe076e0e8
Successfully built pyIUDX
  Found existing installation: pyIUDX 0.0.1
    Uninstalling pyIUDX-0.0.1:
      Successfully uninstalled pyIUDX-0.0.1


## Accessing Catalogue server
All sensor properties are stored in the catalogue server. Querying the catalog entry for a given sensor provides us metadata and information about the sensor, particularly:
* The sendor id (the unique id with which we query the sensor information and data)
* Tags associated with the sensor
* Information on the type of data provided by the sensor and their units

### Import *cat* class from *pyIUDX.cat*
The *cat* class provides the APIs to fetch data from the catalogue server.

In [0]:
from pyIUDX.cat import cat

In [0]:
# Specify the catalogue server details.
# initialize a catalogue class
cat = cat.Catalogue("https://pudx.catalogue.iudx.org.in:443/catalogue/v1")

In [5]:
print(cat)

### Search for catalogue items (sensors)
*getManyResourceItems* member of the catalogue class can be used to fetch a filtered version of catalogue items. The example below shows how to obtain items (sensors) whose tags attribute have the values "aqi" or "aqm". The metadata is returned as a list of dictionary item for each sensor.

In [0]:
attributes = {"tags": ["aqi", "aqm"]}
allAQMItems = cat.getManyResourceItems(attributes)

In [7]:
print(allAQMItems)

[{'@context': ['https://raw.githubusercontent.com/iudx/iudx-ld/master/data_models/environment/airQuality/env_aqm_climoPune_0.json'], 'onboardedBy': 'Technical consultant at rbccps.org', 'itemDescription': 'Describes Air Quality Monitoring (AQM) Resource.', 'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/ABC Farm House Junction_4', 'deviceModelInfo': {'type': 'Property', 'value': {'url': 'http://www.boschclimo.com/', 'brand': 'Climo', 'model': 'Bosch-Climo'}}, 'refBaseSchema': {'type': 'Relationship', 'value': 'https://raw.githubusercontent.com/iudx/iudx-ld/master/base_schemas/v0.0.0/resourceItem_schema.json'}, 'itemType': {'type': 'Property', 'value': 'resourceItem'}, 'tags': {'type': 'Property', 'value': ['environment', 'air quality', 'air', 'aqi', 'aqm', 'climo', 'climate', 'pollution', 'so2', 'co2', 'co', 'no', 'no2', 'pm2.5', 'pm25', 'lux', 'pm10', 'humidity', 'temperature', 'ozone', 'o3', 'noise', 'light', 'uv']}, 'resourceServerGrou

In [0]:
allAQMItemsCount = cat.getItemCount(attributes)

In [9]:
print(allAQMItemsCount)

54


### Filter the catalogue response
An unfiltered call to *getManyResourceItems* will return all information associated with each sensor. Using the filters option, we can filter the information returned.

In the example below, by specifying the filters = ["id"], only the "id" of the sensor is returned.

In [0]:
filters = ["id"]
allAQMItemsByID = cat.getManyResourceItems(attributes, filters)

In [11]:
print(allAQMItemsByID)

[{'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/ABC Farm House Junction_4'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Ambedkar society circle_29'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Appa_Balwant_Square_30'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Bajaj Statue Square_50'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Blue Diamond Square (Hotel Taj)_10'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/BopadiSquare_65'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Bremen Square_37'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/BRTS Visharant wadi_38'}, {'id': 'r

### Filter items (sensors) by geo-location
The function *getManyResourceItems* can also be used to filter items (sensors) based on their locations. In the example below, we specify the area of interest as a circle with center specified by latitude ("lat") and longitude ("lon") and radius in meters.
In this case, we are requesting for sensors around a 3km radius.
We also will specify that we are specifically interested in Air Quality Monitoring Stations present in that region.
We only require the ID's for now, which we can obtain by passing a filters option

In [0]:
geo1 = {"circle": {"lat": 18.539107, "lon": 73.903987, "radius": 3000}}
attributes = {"tags": ["aqm"]}
filters = ["id"]


The below call returns all sensors with tags "aqi" or "aqm" and within the geographical area specified. Further, as specified by the filters argument, only the "id" information is returned for each of the sensors.


In [0]:
allAQMItemsByID = cat.getManyResourceItems(attributes, filters, geo=geo1)

In [14]:
print(allAQMItemsByID)
print("Number of items = ", len(allAQMItemsByID))

[{'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Ambedkar society circle_29'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Blue Diamond Square (Hotel Taj)_10'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Dr Baba Saheb Ambedkar Sethu Junction_3'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/ABC Farm House Junction_4'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Gunjan Square_18'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Golf Club Junction_27'}, {'id': 'rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/Vadgaon_Sheri_Square_52'}]
Number of items =  7


## Fetch data from sensors
In this section we will fetch sensor data from the resource server.

We will iterate across the sensors and obtain the required information for each sensor.

### Import *item* class from *pyIUDX.rs*
The *item* class provides the APIs to access relevant sensor data.

In [0]:
# Import the item class from pyIUDX.rs
from pyIUDX.rs import item

### Plot sensor locations

We can pass the previously obtained list of filtered Air Quality Monitoring stations to the Items class. This will load a list of resourceItem objects and provide neat access to their data.

In [16]:
m = folium.Map(location=[18.5204,73.8567],zoom_start=12)
aqms = item.Items("https://pudx.catalogue.iudx.org.in/catalogue/v1", allAQMItemsByID)
for sensor in aqms:
  sensor_id = sensor.id  
  print("Sensor location = ", sensor.location.coordinates)
  folium.Marker([sensor.location.coordinates[1], sensor.location.coordinates[0] ], popup=sensor_id).add_to(m)    
m

Sensor location =  [73.903987, 18.539107]
Sensor location =  [73.879657, 18.5529659]
Sensor location =  [73.8843632, 18.5412778]
Sensor location =  [73.88559, 18.538425]
Sensor location =  [73.8891134, 18.5461542]
Sensor location =  [73.9113707, 18.5583356]
Sensor location =  [73.8893219, 18.5526023]


### Fetch PM2 and PM10 data from the list of sensors
We will iterate across the list of sensors and obtain the latest *PM2_MAX* and *PM10_MAX* values from each sensor. Further, we will also obtain data for a specified duration.

Let's use one of the items form the list of sensors we just created

In [17]:
print(aqms[0].id)

rbccps.org/aa9d66a000d94a78895de8d4c0b3a67f3450e531/pudx-resource-server/aqm-bosch-climo/ABC Farm House Junction_4


We can find out all the QuantitativeProperties (measured properties) of an aqm item by calling the Object.quantitativeProperties property.
Since our previous filter filtered out all AQM sensors, we can assume that the quantitative properties for sensor aqms[0] is the same for the rest.

In [18]:
print(aqms[0].quantitativeProperties)

['AIR_PRESSURE', 'AQI', 'CO2_MAX', 'CO2_MIN', 'CO_MAX', 'CO_MIN', 'HUMIDITY', 'LIGHT', 'NO2_MAX', 'NO2_MIN', 'NO_MAX', 'NO_MIN', 'OZONE_MAX', 'OZONE_MIN', 'PM10_MAX', 'PM10_MIN', 'PM2_MAX', 'PM2_MIN', 'SO2_MAX', 'SO2_MIN', 'TEMPRATURE_MAX', 'UV_MAX', 'UV_MIN']


A quantitativeProperty also has further meta information related to that property, such as a detailed description, units, etc.
We can get a list of such attributes for a quantitativeProperty, and access them directly.
For e.g, for PM10_MAX.


In [19]:
print(aqms[0].CO2_MAX.attributes)
print("Name of the property is " + aqms[0].CO2_MAX.symbol)
print("Units of the property are " + aqms[0].CO2_MAX.symbol)
print("The property tells us the " + aqms[0].CO2_MAX.describes)

{'name': 'CO2_MAX', 'describes': 'Maximum value of CO2 for the last 24 hours', 'symbol': 'ppm', 'unitCode': 'X59', 'unitText': 'part per million (ppm)'}
Name of the property is ppm
Units of the property are ppm
The property tells us the Maximum value of CO2 for the last 24 hours


We need to call the object's latest() method to get the latest data .
Calling aqms.latest() will update the latest values of all these properties and for **all the previously filtered sensors**.

In [20]:
aqms.latest()

Warning: ignored

We can now call aqms[0]."quantitativeProperty".value to obtain a numpy array with the first column as datetime and second column as that property's value.
A quantitativeProperty always has a .value attribute.


In [0]:
aqms[0].CO2_MAX.value

Calling latest() once is enough. We can access other quantitativeProperties as well.

In [0]:
aqms[0].SO2_MAX.value

Supposing there is a need to find the trend of a particular property over a period of time. We can use Items object's during() functionality to show a time series view of that quantitativeProperty. We need to specify the start and end time in utc format. Let's get the data during 27th October 2019 and 30th October 2019.

In [0]:
aqms.during("2019-10-27T00:00:00.000Z", "2019-10-30T00:00:00.000Z")



Now we can repeat what we did earlies and find the value of the quantitativeProperty **during** that period of time for **sensor 0**

In [0]:
print(aqms[0].CO2_MIN.value)

We can plot this right away!

In [0]:
import matplotlib.pyplot as plt

plt.scatter(aqms[0].CO2_MIN.value[:,0], aqms[0].CO2_MIN.value[:,1])